In [1]:
import pandas as pd
import numpy as np
import json
import pandas as pd
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import flux_variability_analysis
import pickle
from tqdm import tqdm 
from ast import literal_eval
import cobra
import matplotlib.pyplot as plt
import numpy as np
from multiprocessing import Pool
from tqdm import tqdm
from functools import partial 
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import MaxNLocator, FormatStrFormatter

import gurobipy
gurobipy.setParam('OutputFlag', 0)
import sys
# sys.path.append('../Script/')
from common import *

# input and output

In [2]:
###input
yeast8U_del2_path = '../../Data/model/Yeast-MetaTwin.yml'
ymdb_uptake_file = '../../Data/database/ymdb/ymdb_uptake.csv'

###output
yeast8U_del2_path = '../../Data/model/Yeast-MetaTwin.yml'

# add uptake

In [12]:
model = cobra.io.load_yaml_model(yeast8U_del2_path)

In [13]:
model

Name,None
Memory address,7f5ef8d84610
Number of metabolites,3199
Number of reactions,7461
Number of genes,2057
Number of groups,0
Objective expression,1.0*r_2111 - 1.0*r_2111_reverse_58b69
Compartments,"cell envelope, cytoplasm, extracellular, mitochondrion, nucleus, peroxisome, endoplasmic reticulum, Golgi, lipid particle, vacuole, endoplasmic reticulum membrane, vacuolar membrane, Golgi membrane, mitochondrial membrane"


In [3]:
ymdb_uptake = pd.read_csv(ymdb_uptake_file)
print(ymdb_uptake.shape)
ymdb_uptake.head(1)

(51, 5)


,ID,Name,SMILES,Description,inchikey0
0,YMDB02301,Benzoic acid,OC(=O)C1=CC=CC=C1,"Benzoic acid, C6H5COOH, is a colourless crysta...",WPYMKLBDIGXBTP


# add YMDB id and YMDB name

In [15]:
metabolites = {}
num = 0
for index, row in ymdb_uptake.iterrows():
    num+=1
    met_id_e = 'snu_'+str(num)
    met_compartment = 'e'
    metabolites[met_id_e] = Metabolite(met_id_e, formula='', name=row['ID'], compartment=met_compartment)

    num+=2
    met_id_c = 'snu_'+str(num)
    met_compartment = 'c'
    metabolites[met_id_c] = Metabolite(met_id_c, formula='', name=row['ID'], compartment=met_compartment)

    reaction = Reaction('rxnu'+str(num))
    reaction.name = 'rxnu'+str(num)
    reaction.subsystem = ''
    reaction.lower_bound = 0.  # This is the default
    reaction.upper_bound = 1000.  # This is the default

    met_e = metabolites.get(met_id_e)
    met_c = metabolites.get(met_id_c)
    reaction.add_metabolites({met_e: -1})
    reaction.add_metabolites({met_c: 1})
    reaction.gene_reaction_rule = ''

    try:
        model.reactions.get_by_id('rxnu'+str(num))
    except:
        model.add_reactions([reaction])    

In [16]:
model

Name,None
Memory address,7f5ef8d84610
Number of metabolites,3301
Number of reactions,7512
Number of genes,2057
Number of groups,0
Objective expression,1.0*r_2111 - 1.0*r_2111_reverse_58b69
Compartments,"cell envelope, cytoplasm, extracellular, mitochondrion, nucleus, peroxisome, endoplasmic reticulum, Golgi, lipid particle, vacuole, endoplasmic reticulum membrane, vacuolar membrane, Golgi membrane, mitochondrial membrane"


In [17]:
cobra.io.save_yaml_model(model, yeast8U_del2_path)